In [4]:
%matplotlib inline
import xml.etree.ElementTree as ET
import utils
from utils import *
from keras.datasets import imdb
import numpy as np
import os
from pymongo import MongoClient
import json
from translate import translator
from langdetect import detect
from random import randrange
from ipywidgets import FloatProgress
from IPython.display import display
import time
import tqdm
import dill
from collections import Counter

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


## read danish articles from mongodb

In [16]:
## db variables
host = "mongodb://localhost:27017"
uid = ""
pwd = ""
client = None
db = None
sentiment_collection = None

# connect to mongo
#
print("############# initializing Mongo DB Client for infomedia articles")
mclient = MongoClient(host)
mdb = mclient.sentiments_db
infomedia_collection = mdb.infomedia

############# initializing Mongo DB Client for infomedia articles


## Read the training dataset from file , if it does not exists then read it from db and save it.

In [37]:
## Global scope variables
GLOBAL_WORDS_LIST = []
WORD_TO_IDX = {}
IDX_TO_WORD = {}
CURRENT_LANGUAGE = "da"
current_dir = os.getcwd()
DATA_DIR = current_dir + "/sentiments/data"
ORIGINAL_DATA = DATA_DIR + "/original"
TRAIN_DATA_DIR = DATA_DIR + "/train"
TRAIN_DATA_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_train_data.p"
TRAIN_LABEL_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_train_labels.p"
VALIDATE_DATA_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_validate_data.p"
VALIDATE_LABEL_FILE = DATA_DIR + "/" +CURRENT_LANGUAGE+"_validate_labels.p"
MODEL_PATH = current_dir + "/sentiments/models/"
WORD_TO_IDX_DATA_FILE = DATA_DIR + "/"+CURRENT_LANGUAGE+"_words_to_idx.p"
IDX_TO_WORD_DATA_FILE = DATA_DIR + "/"+CURRENT_LANGUAGE+"_idx_to_words.p"
print DATA_DIR, ORIGINAL_DATA, TRAIN_DATA_DIR
# maximum num of words the sentiment can process
MAX_WORD_COUNT = 500
vocabsize = 50000
RESERVED_PROCESS_TERM_IDX = vocabsize
DB_RECORD_EXTRACTION_LIMIT = 50000
MODEL_NAME = "model-ln-"+CURRENT_LANGUAGE+"-s"+str(vocabsize)+"-v2017-19-15"+".h5"
print MODEL_NAME

/home/ubuntu/nbs/sentiments/data /home/ubuntu/nbs/sentiments/data/original /home/ubuntu/nbs/sentiments/data/train
model-ln-da-s50000-v2017-19-15.h5


## Steps Training:
1. Read the articles with process terms and its sentiments.
     1.a) the output will be an array of dict items
          [{'id':'4s23s32', 'process_terms':'bestbuy', text:'bestbuy is better than staples', 'sentiment':0.1},
           {'id':'4s23s32', 'process_terms':'staples', text:'bestbuy is better than staples', 'sentiment':1}]
     
2. Detect text language and translate into english
3. Convert all words into index array. 
4. Split data into training and validation set. 
5. Fit the model


In [10]:
def load_training_data_from_db(start=0, limit_size=10):
    results = []
    total = mdb.infomedia.find({}).count()
    print("******** Total records in db = "+str(total))
    cursor = mdb.infomedia.find({}).skip(0).limit(DB_RECORD_EXTRACTION_LIMIT)
#     f = FloatProgress(min=0, max=total)
#     display(f)
    for doc in cursor:
        # print(doc)
#         f.value += 1
        r = {'id': str(doc.get('ArticleId')),
             'text': doc.get('BodyText').encode('utf-8').lower().strip(),
             'process_term': [],
             'sentiment':0.0
             }
#         print(r)
        process_terms= doc.get('kw')
        scores = doc.get('score')
        if process_terms != None:
            for index,item in enumerate(process_terms):
                rcopy = r.copy()
                for x in item:
                    pitem = x.encode("utf-8").lower()
                    rcopy['process_term'].extend([y for y in pitem.split(',')])
                rcopy['process_term'] = set(rcopy['process_term'])   
                rcopy['sentiment'] = scores[index]
                results.append(rcopy)
    
#     print("found db entries = "+results.length())
#     print results[:10]
    print("******** Total records extracted from db = "+str(len(results)))
    
    return results

#get_articles_from_db();

In [11]:
def build_ln_word_idx_list(target_language, src_text):
    global GLOBAL_WORDS_LIST
    text= src_text.lower()
#     text = src_text.decode('utf-8')
#     source_ln = 'en'
#     try:
#         source_ln = detect(text)
# #         source_ln = 'da'
#     except:
#         print("Failed to detect lang for text = "+text[:30])
#         return
        
# #     print(" detected language = "+source_ln)
#     if source_ln == target_language:
    words = text.split()
    GLOBAL_WORDS_LIST.extend(words)
#         GLOBAL_WORDS_LIST.extend(x for x in words if x not in GLOBAL_WORDS_LIST)

In [12]:
def convert_words_to_idx(text, process_terms):
    global WORD_TO_IDX
    word_idx = []
    p_terms = [x.lower().replace('*','') for x in process_terms]
    text = text.lower()
    for word in text.split():
        if word in p_terms:
            word_idx.append(RESERVED_PROCESS_TERM_IDX)
        else:
            if (word not in WORD_TO_IDX):
                word_idx.append(vocabsize - 1)
#                 print ("word not found in vocab = "+word)
            else:
                word_idx.append(WORD_TO_IDX[word])
    
    return word_idx

In [13]:
def load_words_index():
    global GLOBAL_WORDS_LIST
    global WORD_TO_IDX
    global IDX_TO_WORD
    ## 
    # Build the word to idx and idx to word and dump it for later use
    #
    print("***** total words found = "+str(len(GLOBAL_WORDS_LIST)))
    words_occurence = Counter(GLOBAL_WORDS_LIST)
    print("***** total unique words = "+str(len(words_occurence)))
    sorted_words = [w for w,c in words_occurence.most_common(vocabsize-2)]
    print ("sorted word = ")
    print(sorted_words[:10])
    WORD_TO_IDX = {w:idx for idx,w in enumerate(sorted_words)}
    IDX_TO_WORD = {idx:w for idx,w in enumerate(sorted_words)}
    print ("word to idx size = "+str(len(WORD_TO_IDX)))        
    print("id to word size = "+str(len(IDX_TO_WORD)))

In [28]:
def setup_training_data():
    global GLOBAL_WORDS_LIST
    global WORD_TO_IDX
    global IDX_TO_WORD
    results = load_training_data_from_db()
    train_data = []
    train_labels = []
    train_process_terms = [[]]
    word_list = []
    GLOBAL_WORDS_LIST = []
    
    f = FloatProgress(min=0, max=len(results))
    display(f)
    
    ##
    # Build a word list from all the training data
    #
    for rec in results:
        f.value += 1
        build_ln_word_idx_list(CURRENT_LANGUAGE, rec['text'].lower())
    
    #
    # load the words index
    #
    load_words_index()
    
    #    
    # convert the train_data words to indexes
    #
    print("*** converting texts to ids")
    f.value = 0
    for rec in results:
        f.value += 1
        train_data.append(convert_words_to_idx(rec['text'].lower(),rec['process_term']))
        train_labels.append(rec['sentiment'])
    
    #
    # pick random items from list for validation set
    #
    validate_data = []
    validate_labels = []
    val_set_size = int(round((len(train_data) * 0.3), -1))
    print("validation set size = "+str(val_set_size))
    for i in range(0,val_set_size):
        random_index = randrange(i,len(train_data))
        validate_data.append(train_data[random_index])
        validate_labels.append(train_labels[random_index])
        del train_data[random_index]
        del train_labels[random_index]
       
    print(results[1])
    print(train_data[1])
    print(train_labels[1])
    print(validate_data[1])
    print(validate_labels[1])
    print("validation data size = "+str(len(validate_data)))
    print("validation labels size = "+str(len(validate_labels)))
    print("train data size = "+str(len(train_data)))
    print("train labels size = "+str(len(train_labels)))
    #
    # save it for later use. 
    #
    dill.dump(GLOBAL_WORDS_LIST, open(WORD_TO_IDX_DATA_FILE, "wb" ))
    print("***** saved GLOBAL_WORDS_LIST to pickle files..")
    dill.dump(train_data, open(TRAIN_DATA_FILE, "wb" ))
    print("***** saved train_data to pickle files..")
    dill.dump(train_labels, open(TRAIN_LABEL_FILE, "wb" ))
    print("***** saved train_labels to pickle files..")
    dill.dump(validate_data, open(VALIDATE_DATA_FILE, "wb" ))
    print("***** saved validate_data to pickle files..")
    dill.dump(validate_labels, open(VALIDATE_LABEL_FILE, "wb" ))
    print("***** saved validate_labels to pickle files..")
    
    print("***** SAVED ALL FILES.....")
   


setup_training_data()

******** Total records in db = 59641
******** Total records extracted from db = 65018
***** total words found = 28615201
***** total unique words = 661484
sorted word = 
['i', 'og', 'at', 'er', 'det', 'en', 'til', 'p\xc3\x83\xc2\xa5', 'for', 'af']
word to idx size = 49998
id to word size = 49998
*** converting texts to ids
validation set size = 19510
{'process_term': set(['semler: audi', 'audi*', 'semler holding a/s']), 'text': 'de tyske bilproducenter og is\xc3\x83\xc2\xa6r mercedes klarer sig godt i den nye tuv report 2017.  den store overraskelse er, at to nyere modeller fra kia, der giver syv \xc3\x83\xc2\xa5rs garanti p\xc3\x83\xc2\xa5 teknikken, klarer sig d\xc3\x83\xc2\xa5rligst i deres klasse. unders\xc3\x83\xc2\xb8gelse af cirka ni millioner i tyskland ligger til grund for rapporten.    fakta de bedste   de bedste indtil tre \xc3\x83\xc2\xa5r mercedes glk 2,1 porsche 911 carrera 2,1 mercedes b-klasse 2,2 mercedes a-klasse 2,3 mercedes slk 2,4 de bedste indtil fem \xc3\x83\xc2\

In [29]:
def load_training_data_from_file():
    global GLOBAL_WORDS_LIST
    global WORD_TO_IDX
    global IDX_TO_WORD
    GLOBAL_WORDS_LIST = []
    
    WORD_TO_IDX = {}
    IDX_TO_WORD = {}
    GLOBAL_WORDS_LIST = pickle.load(open( WORD_TO_IDX_DATA_FILE, "rb" ))
    
    #
    # load training data from file
    #
    train_data = pickle.load(open(TRAIN_DATA_FILE, "rb" ))
    train_labels = pickle.load(open(TRAIN_LABEL_FILE, "rb" ))
    validate_data = pickle.load(open(VALIDATE_DATA_FILE, "rb" ))
    validate_labels = pickle.load(open(VALIDATE_LABEL_FILE, "rb" ))
     #
    # load the words index
    #
    load_words_index()
    
    return train_data,train_labels, validate_data, validate_labels

## Training Code
1. load the traing and validation data from disk
2. Pad the data to match the shape of the Cnv
3. Prepare the model

In [30]:
def get_padded_data(word_idx):        
    return sequence.pad_sequences(word_idx,maxlen=MAX_WORD_COUNT, value=0)

In [32]:
t_data, t_labels, v_data, v_labels = load_training_data_from_file()
print("===================================================================================")
print WORD_TO_IDX[u'konkurrence']
print IDX_TO_WORD[WORD_TO_IDX[u'konkurrence']]
print len(WORD_TO_IDX)
index = randrange(0,len(t_data))
print("===================================================================================")
print(t_data[index:index+1])
print("-------------------------------")
print(t_labels[index:index+1])
print("-------------------------------")
print("validation data size = "+str(len(v_data)))
print("validation labels size = "+str(len(v_labels)))
print("train data size = "+str(len(t_data)))
print("train labels size = "+str(len(t_labels)))



# print(pt[index:index+1])

***** total words found = 28615201
***** total unique words = 661484
sorted word = 
['i', 'og', 'at', 'er', 'det', 'en', 'til', 'p\xc3\x83\xc2\xa5', 'for', 'af']
word to idx size = 49998
id to word size = 49998
1054
konkurrence
49998
[[80, 21, 18, 156, 1203, 33, 13351, 3, 793, 43779, 8481, 0, 15, 396, 7823, 22, 496, 56, 565, 181, 76, 2, 7969, 741, 5, 118, 981, 0, 49999, 4, 354, 417, 389, 109, 9, 11, 2482, 8859, 0, 98, 49999, 46, 49999, 75, 11, 3911, 2800, 0, 1605, 3, 4, 87, 461, 848, 10, 691, 2, 59, 7969, 12, 1020, 8, 33122, 1450, 3, 83, 6473, 1, 3839, 0, 49999, 8238, 49999, 16, 895, 131, 2, 236, 56, 9, 2438, 3717, 2, 11, 12, 60, 1919, 8, 5, 49999, 0, 49999, 82, 1284, 474, 2, 5, 2366, 2185, 7, 1191, 20, 540, 0, 20604, 9, 528, 1, 2854, 41, 11, 3568, 49999, 26, 528, 1, 565, 8921, 6934, 0, 49999, 41, 528, 4958, 21, 5, 443, 4024, 42, 11, 11587, 14, 49999, 33, 565, 42, 11587, 14, 49999, 4, 29, 281, 2482, 6230, 1142, 1, 5374, 4, 3, 49999, 1, 49999, 49999, 10, 741, 1436, 3385, 49999, 1476, 0,

In [34]:
pad_train_data = get_padded_data(t_data)
pad_validation_data = get_padded_data(v_data)
print len(pad_train_data)
print len(pad_validation_data)
print pad_train_data.shape
print pad_validation_data.shape
v_bin_labels = [1 if x>=0 else 0 for x in v_labels]
t_bin_labels = [1 if x>=0 else 0 for x in t_labels]
print v_bin_labels[:20]
print t_bin_labels[:20]
print(Counter(v_labels))
print vocabsize
print MAX_WORD_COUNT
print pad_train_data[0]

45508
19510
(45508, 500)
(19510, 500)
[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Counter({1: 8514, 0: 8028, -1: 1977, 0.5: 606, -0.5: 385})
50000
500
[    0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  

In [35]:
model = Sequential()
model.add(Embedding(input_dim=vocabsize+1, output_dim=32, input_length=MAX_WORD_COUNT, dropout=0.2))
model.add(Dropout(0.2))
model.add(Convolution1D(64, 5, activation='relu', border_mode='same'))
model.add(Dropout(0.5))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer=Adam(),  metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 500, 32)       1600032     embedding_input_4[0][0]          
____________________________________________________________________________________________________
dropout_10 (Dropout)             (None, 500, 32)       0           embedding_4[0][0]                
____________________________________________________________________________________________________
convolution1d_4 (Convolution1D)  (None, 500, 64)       10304       dropout_10[0][0]                 
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 500, 64)       0           convolution1d_4[0][0]            
___________________________________________________________________________________________

In [36]:
model.fit(pad_train_data,t_bin_labels,validation_data=(pad_validation_data, v_bin_labels), nb_epoch=5, batch_size=64)

Train on 45508 samples, validate on 19510 samples
Epoch 1/5
45508/45508 [==============================] - 25s - loss: 0.2977 - acc: 0.8735 - val_loss: 0.2369 - val_acc: 0.9133
Epoch 2/5
45508/45508 [==============================] - 24s - loss: 0.2075 - acc: 0.9117 - val_loss: 0.2063 - val_acc: 0.9313
Epoch 3/5
45508/45508 [==============================] - 25s - loss: 0.1684 - acc: 0.9334 - val_loss: 0.1952 - val_acc: 0.9301
Epoch 4/5
45508/45508 [==============================] - 24s - loss: 0.1468 - acc: 0.9409 - val_loss: 0.2016 - val_acc: 0.9158
Epoch 5/5
45508/45508 [==============================] - 24s - loss: 0.1311 - acc: 0.9464 - val_loss: 0.2154 - val_acc: 0.9175


In [38]:
model.save_weights(MODEL_PATH + MODEL_NAME)

## Test / Predict

In [39]:
model.load_weights(MODEL_PATH + MODEL_NAME)

In [46]:
text="Der er opstået en fejl: Beskeden er sendt. * ) skal udfyldes.  Næsten hver anden firmabil er en Mercedes, BMW eller Audi. Andelen voksede i fjor, og tendensen ser ud til at fortsætte. De danske firmabilister er vilde med jobbiler, der har fire ringe, brede nyrer eller en stjerne på snuden, og andelen vokser fortsat. Mens Audi, BMW og Mercedes i 2015 med 5479 stk. stk. udgjorde 44,6 pct. i den vigtige klasse af erhvervsleasede firmabiler i den såkaldte D-størrelse, var andelen i fjor med 6461 stk. på 45,7 pct.  Især Mercedes, der har skruet ned for priserne og op for markedsføringskanonerne, og har lanceret nye modeller som E-klassen, er gået frem. Og hvis vi tæller modeller som Volvo S60/V60 og VW Passat med, der ligger på kanten af at kunne kalde sig premium, er der ikke meget tilbage til de andre mærker.  Der er flere årsager til fremgangen for premiummærkerne Audi, Mercedes og BMW, vurderer branchefolk, vi har talt med. De tror, at tendensen vil fortsætte i 2017. Mads Iversen,der er operations director hos Leaseplan, peger på den hårde konkurrence:  - Audi, Mercedes og BMW har alle kørt med attraktive kampagner, og de er presset af Volvo. Og så er det alle biler med en høj restværdi, så de samlede udgifter, den såkaldte TCO, er lav, siger han.  Thomas Lindgren Mortensen, som er adm. direktør for Alm. Brand Leasing, pointerer, at det i store træk ikke er dyrere for firmaet at vælge Mercedes frem for en Ford Mondeo, fordi den samlede TCO er den samme. Brugeren skal så vurdere, om vedkommende vil beskattes højere. Og det vælger stadig flere, især i takt med, at premiummærkerne har sænket de beskatningsmæssige værdier, så forskellen ikke er så stor.  F.eks. kan du hos Nordania Leasing få en Ford Mondeo stationcar med Titaniumudstyr, 180 hk dieselmotor og automatgear til en beskatningsværdi på 368.000 kr. mens en BMW 3-serie stationcar med 190 hk dieselmotor og automatgear samt businessudstyrspakke ligger på 404.000 kr. Hvis du kan nøjes med manuelt gear, er beskatningsværdien 374.000 kr.  Den nye Mercedes E-klasse starter fra en beskatningsmæssig værdi på 520.000 kr. for en 220 D sedan Business 9gtronic med automatgear og 194 hk hos Nordania Leasing. Her følger den nye BMW 5-serie trop med beskatningsværdier fra 510.000 kr. for en 520d sedan aut. med 190 hk hos BMW Finans.  Samtidig har importørerne udvidet viften af muligheder. BMW markedsfører også 3-serie GT, 4-serien og 4-serie Coupé, som ellers tidligere har været nichemodeller. Det giver udslag, også hos Leaseplan.  - Vi har kunder, der er kommet ind efter en Ford Mondeo eller Opel Insignia, men er endt med at køre i BMW 3-serie, en Audi A4 eller en Mercedes C-klasse. Det er blevet mere legalt at køre BMW. Faren er selvfølgelig også, at det ikke er så eksklusivt som tidligere, men det er det op til bilfabrikkerne at vurdere konsekvensen af, siger Mads Iversen fra Leaseplan.  Hos Alm. Brand ser direktør Thomas Lindgren Mortensen også i stigende grad, at flere firmaer giver grønt lys til, at medarbejderne må køre i premiummærker - og i særdeleshed Mercedes:  - Tidligere så vi en del virksomheder, som havde en politik om, at man ikke måtte køre i Mercedes. Men det er ved at ændre sig mange steder, konstaterer ThomasLindgren Mortensen, som tror, at viften bliver bredere, så et mærke som Jaguar vil få mere fat.  Her har det engelske mærke på det seneste skruet op for markedsføringen af både XE, XF, og den nye SUV, F-Pace, som er kommet godt fra land:  - Det har været med Jaguar som med Mercedes tidligere. Men i dag er der ikke noget ekstravagant i at køre Jaguar, så det mærke vil vi se mere til, mener Thomas Lindgren Mortensen.  Men også nedsættelse af bilafgiften i henholdsvis 2015 og 2016, hvor de beskatningsmæssige værdier faldt med henholdsvis omkring en snes og en halv snes tusinde kr., mens brugerne stadig har den samme ramme at købe bil for, har medført et større rådighedsbeløb og dermed ændringer. Thomas Lindgren Mortensen kan se, at afgiftsnedsættelserne har rykket ved købsmønstret:  - Vi ser, at folk bruger de ekstra penge til enten at købe ekstraudstyr eller købe en dyrere bil, fordi afstanden er blevet mindre. De vælger ikke at spare pengene og måske gå en bilklasse ned. Så det er et godt argument for at sænke afgiften yderligere, siger Thomas Lindgren Mortensen, som tror, at de, som nu har valgt et premiummærke, holder fast.  - Når du først har vænnet dig til at køre en bil fra et premiummærke, er det svært at gå tilbage, siger han.  De nye muligheder med større rådighedsbeløb smitter af på flere fronter, lyder meldingen fra Mads Iversen fra Leaseplan.  - Firmabilisterne har fået videre rammer at købe bil for, og de penge bruger de især på sikkerhedsudstyr som f.eks. adaptiv fartpilot eller full LED-lys. Det ligger fint i tråd med virksomhedernes ønsker om mere sikre arbejdspladser og forøger samtidig bilernes gensalgsværdi. Premiummærkerne har været i stand til at sammensætte udstyrspakker med disse sikkerhedselementer til attraktive priser, hvilket f.eks. Opel og Ford ikke har haft samme muligheder for.  - Vi vil også hellere sælge en brugt bil med en stor sikkerhedspakke frem for f.eks. læderinteriør, store alufælge eller indbyggede dvd-skærme, siger Mads Iversen.  Han tror lige som Thomas Lindgren Mortensen, at udviklingen fortsætter i 2017:  - Premiummærkerne vil holde samme andel og måske få en lille stigning, hvis tendensen holder, og der ikke sker ændringer. Premiummærkerne holder øje med hinanden. Hvis Audi, Mercedes og BMW er skarpe på tilbud med især sikkerhedsudstyr som for eksempel adaptiv fartpilot, automatisk linjevogter og alarmer for blinde vinkler, vil udviklingen kunne fortsætte. Men de vil også blive presset af Volvo, som er på vej med en ny XC40, hvor prisen efter sigende skulle blive skarp, siger Mads Iversen, der også forudser en fortsat fremgang for crossover/suv-segmentet, hvor Nissan Qashqai har godt fat, og nyheder som bl.a. Peugeot 3008, Peugeot 5008 og Seat Altea nu for alvor gør deres indtog i sammen med VW Tiguan"
proc_terms=[
    "Econic, Smart, Unimog, Viano, Vito, Mercedes-Benz*, Mercedes*, Daimler",
        "Econic, Smart, Unimog, Viano, Vito, Mercedes-Benz*, Mercedes*, Daimler",
        "brand, alm brand*, Alm.brand, Alm. Brand",
        "Audi*",
        "jaguar",
        "Jaguar, Land Rover, Range Rover, Landrover*",
        "Jaguar, Land Rover, Range Rover, Landrover*"

    ]

In [47]:
#text_clean = re.sub('\W+', ' ', text)
textWordsIdxs = convert_words_to_idx(text,proc_terms)
textIdxArrayPadded = get_padded_data([np.array(textWordsIdxs)])
prediction = model.predict(textIdxArrayPadded, batch_size=1,verbose=1)
print prediction

1/1 [==============================] - 0s
[[ 0.242]]
